# Task two

In task two we are looking at Pacific Sea Surface Temperature (SST) and local forecasts.
The two datasets we will be using for this are:

* the global SST data used in week four (1960-2016)
* the SST data recoreded at at Port Vila (Vanuatu) between 1993 and 2021.

We will predominantly use pricipal component analysis to develop a model based on a subset of the global SST data.
The subset of global SST data used will be of the Pacific region that includes the station I have chosed (Port Vila Vanuatu)
To test the PC model we then try and predict the temperature recorded at the location chosen in the Pacific region and compare the outputs with the recorded data.

## Part one

First we analyse the data from the global SST data.
This data is **describe data here**.

In [32]:
using Pkg
Pkg.activate("..")
using ClimateBase, Plots, Statistics, Glob

  Activating environment at `~/Documents/GitHub/MATH5271/Project.toml`


In [68]:
SST_datapath = joinpath("../..", "globalSST/sst")
globalSST_data = glob(joinpath(SST_datapath, "*.nc"))
globalSST_file = NCDataset(globalSST_data; aggdim = "time")
#SSTglobal = ClimArray(globalSST_file)

MethodError: MethodError: no method matching ClimArray(::NCDatasets.MFDataset{NCDatasets.DeferDataset, 1, String, NCDatasets.DeferAttributes, NCDatasets.DeferDimensions, NCDatasets.DeferGroups}, ::NCDatasets.MFDimensions{NCDatasets.DeferDimensions})